# Testing Retrieval-Augmented Generation (RAG) with Amazon Bedrock

This notebook demonstrates:

- Implementation of RAG techniques using Amazon Bedrock
- Enhancement of LLMs with external knowledge
- Utilization of custom knowledge bases
- Testing of retrieval functionality

Ideal for developers and AI enthusiasts exploring how to augment language models with domain-specific knowledge for improved accuracy and relevance.


⭐ Do you find this project usefull? Please consider giving it a star on GitHub! 

💖 Made with dedication to help fellow developers. Your support means the world! 

In [1]:
import boto3
import pprint

In [2]:
# import kb_id from my private file config.py
from config import *

<a id="test-a-knowledge-base"></a>
## AWS Bedrock Agent with Knowledge Base Integration

This script demonstrates how to use AWS Bedrock Agent Runtime to perform RAG (Retrieval-Augmented Generation) operations with a custom knowledge base. It includes:

- Setup for Bedrock Agent Runtime client
- A function to query the LLM with knowledge base support
- Example usage with Claude 3 Sonnet model
- Handling of session IDs for follow-up questions
- Citation retrieval from the knowledge base

🚀 Ideal for developers working on AI-powered Q&A systems or chatbots with custom knowledge bases.

In [8]:
pp = pprint.PrettyPrinter(indent=2)
region_name = 'us-east-1'
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name = region_name)

In [7]:
model_id =  ["Claude 3 Sonnet", "anthropic.claude-3-sonnet-20240229-v1:0"]
kb_id = kb_id

In [4]:
def ask_bedrock_llm_with_knowledge_base(query: str, model_arn: str, kb_id: str, sessionId:str = None) -> str:
    response = "None"
    # If this is the first RAG action
    if not sessionId:
        response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={
                'text': query
            },
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kb_id,
                    'modelArn': model_arn
                }
            },
        )
    else:
        # If sessionId is present
        response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={
                'text': query
            },
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kb_id,
                    'modelArn': model_arn
                }
            },
            sessionId = sessionId
        )       

    return response

In [5]:
query = 'list 5 insignificant packages listted in the exported environments'


model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id[1]}'
response = ask_bedrock_llm_with_knowledge_base(query, model_arn, kb_id)
# !important: keep the same session Id for the follow up questions
sessionId = response['sessionId']
generated_text = response['output']['text']
# the list of citations from the knowledge base
citations = response["citations"]
contexts = []

# extra: print the citations from the knowledgebase
#for citation in citations:
#    retrievedReferences = citation["retrievedReferences"]
#    for reference in retrievedReferences:
#        contexts.append(reference["content"]["text"])
print(f"---------- Generated using {model_id[0]}:")
pp.pprint(generated_text)
print(f'---------- The citations for the response generated by {model_id[0]}:')
pp.pprint(contexts)
print()

---------- Generated using Claude 3 Sonnet:
('Here are 5 relatively insignificant packages listed in the exported '
 'environments:\n'
 '\n'
 '1. xsel (a command-line program for manipulating the X Window selection)\n'
 '2. xorg-libxmu (X.Org X11 miscellaneous utility library)\n'
 '3. xorg-libxt (X.Org X11 toolkit intrinsics library)\n'
 '4. xorg-libice (X.Org Inter-Client Exchange library)\n'
 '5. xorg-libxau (X.Org X11 authorisation library)')
---------- The citations for the response generated by Claude 3 Sonnet:
[]



<a id="test-persistent-conversation-with-rag"></a>
## Persistent Conversation Testing with AWS Bedrock Agent

This section demonstrates how to maintain context in a conversation using AWS Bedrock Agent Runtime:

- Reuses the previous session ID for continuity
- Sends a follow-up query to test conversation persistence
- Processes and displays the response, including any citations

🧠 Showcases the AI's ability to remember and reference previous interactions within the same session.

In [6]:
# we will use the same code as before, but keeping the same sessionId
query2 = 'what did you told me in the last response?'


model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id[1]}'
response = ask_bedrock_llm_with_knowledge_base(query2, model_arn, kb_id,sessionId=sessionId)
# !important: keep the same session Id for the follow up questions
sessionId = response['sessionId']
generated_text = response['output']['text']
citations = response["citations"]
contexts = []

# extra: print the citations from the knowledgebase
#for citation in citations:
#    retrievedReferences = citation["retrievedReferences"]
#    for reference in retrievedReferences:
#        contexts.append(reference["content"]["text"])
print(f"---------- Generated using {model_id[0]}:")
pp.pprint(generated_text)
print(f'---------- The citations for the response generated by {model_id[0]}:')
pp.pprint(contexts)
print()

---------- Generated using Claude 3 Sonnet:
('Based on the search results, here are 5 packages that seem relatively '
 'insignificant or small in scope that are listed in the exported '
 'environments:\n'
 '\n'
 '1. pluggy (version 1.5.0)\n'
 '2. ptyprocess (version 0.7.0)\n'
 '3. pure_eval (version 0.2.3)\n'
 '4. pygments (version 2.18.0)\n'
 '5. pymsgbox (version 1.0.9)')
---------- The citations for the response generated by Claude 3 Sonnet:
[]



In [9]:
# now we will use metadata filtering

In [10]:
# we will filter based on:
# x-amz-bedrock-kb-source-uri
# the values are:
# s3://mainbucketrockhight5461/test/knowledge-bases/companies/hvfbnbgjhfkdhvjkbnfdbgvjhkdfsgjk.txt
# s3://mainbucketrockhight5461/test/knowledge-bases/companies/vbfhdsjkvbhdfkjsvjfdsbvhjfdsbvfdshjkvbdfsjh.txt
# s3://mainbucketrockhight5461/test/knowledge-bases/companies/medisync.txt
# query:
# "is a leading healthcare technology company"

In [19]:
query = "GreenLeaf"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        'text': query
    },
    retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': KBID_2,
            'modelArn': model_arn,
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'numberOfResults': 20
                }
            }
        }
    }
)

In [20]:
response

generated_text = response['output']['text']
# the list of citations from the knowledge base
citations = response["citations"]
contexts = []

# extra: print the citations from the knowledgebase
#for citation in citations:
#    retrievedReferences = citation["retrievedReferences"]
#    for reference in retrievedReferences:
#        contexts.append(reference["content"]["text"])
print(f"---------- Generated using {model_id[0]}:")
pp.pprint(generated_text)
counter = 0
for citation in citations:
    counter += 1
    print(f"citation {counter}: \n{citation}")
pp.pprint(contexts)
print()

---------- Generated using Claude 3 Sonnet:
('GreenLeaf Technologies is a leading company in the sustainable energy '
 'sector, founded in 2008 by Dr. Amelia Chen and Dr. Marcus Hoffman. The '
 'company focuses on developing innovative solutions in solar energy '
 'optimization, wind power efficiency, and energy storage solutions with the '
 'mission of accelerating the transition to renewable energy sources. Some of '
 "GreenLeaf's flagship products include:\n"
 '\n'
 '- The SolarMax Pro, an advanced solar panel system with an industry-leading '
 '27% efficiency rate\n'
 '- The WindFlow Turbine series, featuring a proprietary blade design that '
 'increases energy output by up to 40% compared to traditional models\n'
 '- The EcoCell, a revolutionary energy storage system that uses organic '
 'compounds and nanotechnology to create a highly efficient and '
 'environmentally friendly battery GreenLeaf has experienced rapid growth, '
 'reporting annual revenues of $3.2 billion in 2020, a

In [ ]:
# now we will use metadata filtering to only use the file s3://mainbucketrockhight5461/test/knowledge-bases/companies/medisync.txt

In [17]:
query = "GreenLeaf"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        'text': query
    },
    retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': KBID_2,
            'modelArn': model_arn,
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'filter': {
                        'equals': {
                            'key': 'x-amz-bedrock-kb-source-uri',
                            'value': 's3://mainbucketrockhight5461/test/knowledge-bases/companies/medisync.txt'
                        }
                    },
                    'numberOfResults': 20
                }
            }
        }
    }
)

In [18]:
response

generated_text = response['output']['text']
# the list of citations from the knowledge base
citations = response["citations"]
contexts = []

# extra: print the citations from the knowledgebase
#for citation in citations:
#    retrievedReferences = citation["retrievedReferences"]
#    for reference in retrievedReferences:
#        contexts.append(reference["content"]["text"])
print(f"---------- Generated using {model_id[0]}:")
pp.pprint(generated_text)
counter = 0
for citation in citations:
    counter += 1
    print(f"citation {counter}: \n{citation}")
pp.pprint(contexts)
print()

---------- Generated using Claude 3 Sonnet:
('GreenLeaf Technologies is a major competitor of MediSync Solutions in the '
 'healthcare technology sector. Both companies are leading players in '
 'developing innovative digital solutions for healthcare, including electronic '
 'health records, telemedicine platforms, AI-driven diagnostic tools, and '
 'wearable medical devices. GreenLeaf has made significant progress in areas '
 'like AI for early cancer detection through its GreenMind division, and its '
 "VitaTrack wearable device is seen as a strong competitor to MediSync's "
 'MediVital smartwatch. The two companies are engaged in an intense rivalry '
 'across various fronts, including research and development investment, '
 'expansion into new markets, talent acquisition, and corporate social '
 'responsibility initiatives. In terms of financial performance, GreenLeaf '
 "reported revenues of $1.6 billion in 2021, slightly behind MediSync's $1.8 "
 'billion. However, the gap between